In [1]:
!/usr/bin/python3 -m pip install --force-reinstall ./../../../target/wheels/biodivine_aeon-0.0.9a4-cp37-abi3-macosx_11_0_arm64.whl

Defaulting to user installation because normal site-packages is not writeable
Processing /Users/smijeva/PycharmProjects/biodivine-aeon-py/target/wheels/biodivine_aeon-0.0.9a4-cp37-abi3-macosx_11_0_arm64.whl
  Attempting uninstall: biodivine-aeon
    Found existing installation: biodivine_aeon 0.0.9a4
    Uninstalling biodivine_aeon-0.0.9a4:
      Successfully uninstalled biodivine_aeon-0.0.9a4

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


## Demonstration of control on Myeloid model

The model we will observe is coming from Krumsiek, J., Marr, C., Schroeder, T., and Theis, F. J. (2011). Hierarchical differentiation of myeloid progenitors is encoded in the transcription factor network. PloS one, 6(8), e22649.

It describes myeloid cell differentiation into four possible types - erythrocytes, megakaryocytes, monocytes, and granulocytes.

![Myeloid_differentiation](myeloid.png)

In [2]:
import biodivine_aeon as ba
import pandas as pd

In [3]:
# Load the model and create a Perturbation Graph

model = ba.BooleanNetwork.from_file('myeloid_witness.aeon')
pstg = ba.PerturbationGraph(model)

In [4]:
# Find attractors representing the four possible phenotypes

attractors = ba.find_attractors(pstg.as_original())

erythrocyte = pstg.fix_variable("EKLF", True).vertices()
erythrocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(erythrocyte).cardinality() > 0.0][0]

megakaryocyte = pstg.fix_variable("Fli1", True).vertices()
megakaryocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(megakaryocyte).cardinality() > 0.0][0]

monocyte = pstg.fix_variable("cJun", True).vertices()
monocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(monocyte).cardinality() > 0.0][0]

granulocyte = pstg.fix_variable("Gfi1", True).vertices()
granulocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(granulocyte).cardinality() > 0.0][0]

In [5]:
# Let's demonstrate now, how we can compute source-target control.

# First we need to compute a symbolic structure containing the perurbations and results
sym_results: ba.AttractorControlMap = pstg.one_step_control(erythrocyte_att, megakaryocyte_att, verbose=True)

In [6]:
# Now we can either look up for what colors does a specific perturbation work
sym_results.perturbation_working_colors({"v_EKLF": True})

ColorSet(cardinality=0)

In [7]:
# Or just retrieve all smallest colours with a given minimal robustness to avoid guessing
sym_results.working_perturbations(min_robustness=0.99, verbose=False,return_all=False)

[({'EKLF': False, 'Fli1': True}, ColorSet(cardinality=1))]

In [8]:
# In a similar manner, we can also compute phenotype control. Nonetheless, providing jus target state (or a set of states) is sufficient in this case)

# There are three possible configurations of oscillation
# Forbidden -> attractor where model stabilizes cannot oscillate trhough the phenotype (must stabilize in it)
# Allowed -> attractor where model stabilizes may oscillate
# Required -> attractor where model stabilizes MUST oscillate through the phenotype (i.e. have states both inside and outside of the phenotype space)

# In this demonstration we use forbidden type of oscillation, since myeloid model does not show oscillations and this type is most similar to the source-target control

# Even though the model is small, using ceiled version of the function is recommended, since it allows stop_early functionality, computing the perturbations until 100% robustness is hit or specifying a sensible size_bounc
# phen_sym_results = pstg.phenotype_permanent_control(phenotype=megakaryocyte_att, oscillation="Forbidden", verbose=True)
phen_sym_results = pstg.ceiled_phenotype_permanent_control(
        phenotype=megakaryocyte_att,
        size_bound=11,
        oscillation="Forbidden",
        stop_early=True,
        verbose=True)
phen_sym_results.working_perturbations(min_robustness=0.99, verbose=False,return_all=True)

[0] >> Admissible fixed(Q) sets: 1
>>>>>>>>>>>>> Computing phenotype control with allowed perturbations of cardinality 1


[({'PU1': False, 'Fli1': True}, ColorSet(cardinality=1))]

Trap cardinality 1
Cardinality of phenotype trap set: 1
Inverse trap cardinality 1088
Cardinality of inversed trap set: 1088
Inverse control cardinality 2048
Cardinality of inversed control map: 2048
Cardinality of control map: 0
Best robustness 0 observed in 0 perturbations.
>>>>>> Elapsed: 0ms
Best robustness 0 observed in 0 perturbations.
Robustness 0 achieved for perturbation size 0.
[1] >> Admissible fixed(Q) sets: 11
>>>>>>>>>>>>> Computing phenotype control with allowed perturbations of cardinality 11
Trap cardinality 11
Cardinality of phenotype trap set: 11
Inverse trap cardinality 16768
Cardinality of inversed trap set: 16768
Inverse control cardinality 22528
Cardinality of inversed control map: 22528
Cardinality of control map: 0
Best robustness 0 observed in 0 perturbations.
>>>>>> Elapsed: 1ms
Best robustness 0 observed in 0 perturbations.
Robustness 0 achieved for perturbation size 1.
[2] >> Admissible fixed(Q) sets: 55
>>>>>>>>>>>>> Computing phenotype control with allowe

In [9]:
# Now we can compute a full sets of results for all pairs of source-target control

In [10]:
all_atts = {
    "Erythrocyte": erythrocyte_att,
    "Megakaryocyte": megakaryocyte_att,
    "Monocyte": monocyte_att,
    "Granulocyte": granulocyte_att
}

# Generate al pair possibilities for computing the source-target control
all_att_pairs = [(s,t) for s in all_atts.keys() for t in all_atts.keys() if s != t]

In [11]:
data = {
    'source': [s for s, _ in all_att_pairs],
    'target': [t for _, t in all_att_pairs],
    'onestep': [pstg.one_step_control(all_atts[s], all_atts[t], True).working_perturbations(min_robustness=0.99, verbose=False,return_all=False) for s, t in all_att_pairs],
    'temporary': [pstg.temporary_control(all_atts[s], all_atts[t], True).working_perturbations(min_robustness=0.99, verbose=False,return_all=False) for s, t in all_att_pairs],
    'permanent': [pstg.permanent_control(all_atts[s], all_atts[t], True).working_perturbations(min_robustness=0.99, verbose=False,return_all=False) for s, t in all_att_pairs],
}

df = pd.DataFrame(data)

In [12]:
df

,source,target,onestep,temporary,permanent
0,Erythrocyte,Megakaryocyte,"[({'EKLF': False, 'Fli1': True}, ColorSet(card...","[({'EKLF': False}, ColorSet(cardinality=1)), (...","[({'EKLF': False}, ColorSet(cardinality=1)), (..."
1,Erythrocyte,Monocyte,"[({'PU1': True}, ColorSet(cardinality=1))]","[({'PU1': True}, ColorSet(cardinality=1))]","[({'PU1': True}, ColorSet(cardinality=1))]"
2,Erythrocyte,Granulocyte,"[({'CEBPa': True, 'GATA1': False, 'Gfi1': True...","[({'EgrNab': False, 'CEBPa': True}, ColorSet(c...","[({'CEBPa': True, 'EgrNab': False}, ColorSet(c..."
3,Megakaryocyte,Erythrocyte,"[({'Fli1': False, 'EKLF': True}, ColorSet(card...","[({'EKLF': True}, ColorSet(cardinality=1)), ({...","[({'EKLF': True}, ColorSet(cardinality=1)), ({..."
4,Megakaryocyte,Monocyte,"[({'PU1': True}, ColorSet(cardinality=1))]","[({'PU1': True}, ColorSet(cardinality=1))]","[({'PU1': True}, ColorSet(cardinality=1))]"
5,Megakaryocyte,Granulocyte,"[({'CEBPa': True, 'Fli1': False, 'Gfi1': True,...","[({'CEBPa': True, 'EgrNab': False}, ColorSet(c...","[({'CEBPa': True, 'EgrNab': False}, ColorSet(c..."
6,Monocyte,Erythrocyte,"[({'PU1': False, 'EKLF': True, 'GATA1': True},...","[({'GATA2': True, 'EKLF': True, 'GATA1': True}...","[({'GATA1': True, 'PU1': False, 'EKLF': True},..."
7,Monocyte,Megakaryocyte,"[({'Fli1': True, 'GATA1': True, 'PU1': False},...","[({'Fli1': True, 'PU1': False}, ColorSet(cardi...","[({'Fli1': True, 'PU1': False}, ColorSet(cardi..."
8,Monocyte,Granulocyte,"[({'Gfi1': True, 'EgrNab': False, 'CEBPa': Tru...","[({'CEBPa': True, 'EgrNab': False}, ColorSet(c...","[({'CEBPa': True, 'EgrNab': False}, ColorSet(c..."
9,Granulocyte,Erythrocyte,"[({'GATA1': True, 'EKLF': True, 'PU1': False, ...","[({'GATA2': True, 'GATA1': True, 'EKLF': True}...","[({'GATA1': True, 'EKLF': True, 'PU1': False},..."


In [13]:
for _, r in df.iterrows():
    print(r['source'], "->", r["target"], ':', [x[0] for x in r['permanent']])

Erythrocyte -> Megakaryocyte : [{'EKLF': False}, {'Fli1': True}]
Erythrocyte -> Monocyte : [{'PU1': True}]
Erythrocyte -> Granulocyte : [{'CEBPa': True, 'EgrNab': False}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'cJun': False}]
Megakaryocyte -> Erythrocyte : [{'EKLF': True}, {'Fli1': False}]
Megakaryocyte -> Monocyte : [{'PU1': True}]
Megakaryocyte -> Granulocyte : [{'CEBPa': True, 'EgrNab': False}, {'Gfi1': True, 'CEBPa': True}, {'cJun': False, 'CEBPa': True}]
Monocyte -> Erythrocyte : [{'GATA1': True, 'PU1': False, 'EKLF': True}, {'PU1': False, 'GATA1': True, 'Fli1': False}]
Monocyte -> Megakaryocyte : [{'Fli1': True, 'PU1': False}]
Monocyte -> Granulocyte : [{'CEBPa': True, 'EgrNab': False}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'cJun': False}]
Granulocyte -> Erythrocyte : [{'GATA1': True, 'EKLF': True, 'PU1': False}, {'PU1': False, 'Fli1': False, 'GATA1': True}]
Granulocyte -> Megakaryocyte : [{'Fli1': True, 'PU1': False}]
Granulocyte -> Monocyte : [{'CEBPa': False}]

From this data, we can (manually) derive a following source-target diagram, showing all minimal source-target controls.

![Source_target_controls](myeloid_source_target.png)

In [14]:
df['min_onestep_size'] = df['onestep'].apply(lambda x: len(x[0][0]))
df['min_onestep_options'] = df['onestep'].apply(lambda x: len(x))
df['min_temporary_size'] = df['temporary'].apply(lambda x: len(x[0][0]))
df['min_temporary_options'] = df['temporary'].apply(lambda x: len(x))
df['min_permanent_size'] = df['permanent'].apply(lambda x: len(x[0][0]))
df['min_permanent_options'] = df['permanent'].apply(lambda x: len(x))

In [15]:
df.filter(regex='^(source|target|min_)')

,source,target,min_onestep_size,min_onestep_options,min_temporary_size,min_temporary_options,min_permanent_size,min_permanent_options
0,Erythrocyte,Megakaryocyte,2,1,1,2,1,2
1,Erythrocyte,Monocyte,1,1,1,1,1,1
2,Erythrocyte,Granulocyte,3,1,2,3,2,3
3,Megakaryocyte,Erythrocyte,2,1,1,2,1,2
4,Megakaryocyte,Monocyte,1,1,1,1,1,1
5,Megakaryocyte,Granulocyte,4,3,2,3,2,3
6,Monocyte,Erythrocyte,3,1,3,6,3,2
7,Monocyte,Megakaryocyte,3,1,2,1,2,1
8,Monocyte,Granulocyte,3,1,2,4,2,3
9,Granulocyte,Erythrocyte,4,1,3,6,3,2


In [16]:
# In a similar manner, we can compute phenotype control, even though this time, only target attractor (or phenotype, but in this case we consider attractor to be phenotype) is known
df['phenotype'] = df['target'].apply(lambda t: pstg.ceiled_phenotype_permanent_control(
        phenotype=all_atts[t],
        size_bound=11,
        oscillation="Forbidden",
        stop_early=True,
        verbose=False).working_perturbations(min_robustness=0.99, 
                                             verbose=False,
                                             return_all=False))

In [17]:
df['min_phenotype_size'] = df['phenotype'].apply(lambda x: len(x[0][0]))
df['min_phenotype_options'] = df['phenotype'].apply(lambda x: len(x))

In [18]:
df.filter(regex='^(source|target|min_)')

,source,target,min_onestep_size,min_onestep_options,min_temporary_size,min_temporary_options,min_permanent_size,min_permanent_options,min_phenotype_size,min_phenotype_options
0,Erythrocyte,Megakaryocyte,2,1,1,2,1,2,2,1
1,Erythrocyte,Monocyte,1,1,1,1,1,1,2,1
2,Erythrocyte,Granulocyte,3,1,2,3,2,3,2,3
3,Megakaryocyte,Erythrocyte,2,1,1,2,1,2,3,2
4,Megakaryocyte,Monocyte,1,1,1,1,1,1,2,1
5,Megakaryocyte,Granulocyte,4,3,2,3,2,3,2,3
6,Monocyte,Erythrocyte,3,1,3,6,3,2,3,2
7,Monocyte,Megakaryocyte,3,1,2,1,2,1,2,1
8,Monocyte,Granulocyte,3,1,2,4,2,3,2,3
9,Granulocyte,Erythrocyte,4,1,3,6,3,2,3,2


We can use the phenotype control results to compare whether they are in align with phenotype branching observed in the source of the model

In [19]:
for a in all_atts.keys():
    pd_select = df.loc[df['target'] == a, 'phenotype'].iloc[0]
    print(a,[val[0] for val in pd_select])

Erythrocyte [{'PU1': False, 'Fli1': False, 'GATA1': True}, {'EKLF': True, 'GATA1': True, 'PU1': False}]
Megakaryocyte [{'Fli1': True, 'PU1': False}]
Monocyte [{'PU1': True, 'CEBPa': False}]
Granulocyte [{'cJun': False, 'CEBPa': True}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'EgrNab': False}]


We can also compare the results with the original study

![image](myeloid_phenotype.png)
![image](phenotypes.png)


We can see that the results are consistent, because:

- Phenotype control is very similar to the original study. Since study does not assume all initial states, the control is slightly different and there are some extra variables to control (i.e. min phenotype controls with size greater than 2) but they are not in contradiction with the results. 
- There are some extra possible ways for the phenotype control, esp. negation of the last branch
- Phenotype control is for most of the cases is a super set of the source-targets control. No source-target control is in a contradiction with the phenotype control.
    - This shows that if we know the source and specific target in advance, we can further minimize the size of the control

## Everything below is just WIP

Now let's imagine, that some functions of the original model are unknown

![image](unknowns.png)

In [20]:
# Load the model and create a Perturbation Graph

model_unknown = ba.BooleanNetwork.from_file('myeloid_3unknown.aeon')
pstg_unknown = ba.PerturbationGraph(model_unknown)

In [21]:
# From the original graph, we can see how many colours there are (perturbed contains extra colors representing perturbations)

ba.SymbolicAsyncGraph(model_unknown).unit_colors()
CARDINALITY = ba.SymbolicAsyncGraph(model_unknown).unit_colors().cardinality()
CARDINALITY

2963088.0

In [22]:
# We need to reassign attractor configuration to the new graph
attractors = ba.find_attractors(pstg_unknown.as_original())

erythrocyte = pstg_unknown.fix_variable("EKLF", True).vertices()
erythrocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(erythrocyte).cardinality() > 0.0][0]

megakaryocyte = pstg_unknown.fix_variable("Fli1", True).vertices()
megakaryocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(megakaryocyte).cardinality() > 0.0][0]

monocyte = pstg_unknown.fix_variable("cJun", True).vertices()
monocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(monocyte).cardinality() > 0.0][0]

granulocyte = pstg_unknown.fix_variable("Gfi1", True).vertices()
granulocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(granulocyte).cardinality() > 0.0][0]

all_atts = {
    "Erythrocyte": erythrocyte_att,
    "Megakaryocyte": megakaryocyte_att,
    "Monocyte": monocyte_att,
    "Granulocyte": granulocyte_att
}

# Generate al pair possibilities for computing the source-target control
all_att_pairs = [(s,t) for s in all_atts.keys() for t in all_atts.keys() if s != t]

In [23]:
# An example of the phenotype control computation

phen_sym_results = pstg_unknown.ceiled_phenotype_permanent_control(
        phenotype=megakaryocyte_att,
        size_bound=11,
        oscillation="Forbidden",
        stop_early=True,
        verbose=True)

# Why there is a working phenotype control of 1? Probably attractor got matched to some other state?

[0] >> Admissible fixed(Q) sets: 1
>>>>>>>>>>>>> Computing phenotype control with allowed perturbations of cardinality 2963088
Trap cardinality 367461900
Cardinality of phenotype trap set: 367461900
Inverse trap cardinality 52387260408
Cardinality of inversed trap set: 52387260408
Inverse control cardinality 58027253760
Cardinality of inversed control map: 58027253760
Cardinality of control map: 237920256
>>> {}: 116172; rho = 0.039
Best robustness 0.03920639549011032 observed in 1 perturbations.
>>>>>> Elapsed: 315ms
>>> {}: 116172; rho = 0.039
Best robustness 0.03920639549011032 observed in 1 perturbations.
Robustness 0 achieved for perturbation size 0.
[1] >> Admissible fixed(Q) sets: 11
>>>>>>>>>>>>> Computing phenotype control with allowed perturbations of cardinality 32593968
Trap cardinality 5248294587
Cardinality of phenotype trap set: 5248294587
 Trap non-phenotype progress: 108681 / 586875165400
 Trap non-phenotype progress: 108911 / 586723652404
 Trap non-phenotype progress:

In [24]:
# We can find out the robustness of previously computed perturbations
mega_phen = pstg_unknown.ceiled_phenotype_permanent_control(
        phenotype=megakaryocyte_att,
        size_bound=2,
        oscillation="Forbidden",
        stop_early=False,
        verbose=False)
mega_phen.perturbation_working_colors({"Fli1": True, "PU1": False}).cardinality() / CARDINALITY

1.0

In [25]:
# We can find out the robustness of previously computed perturbations
mega_erythro = pstg_unknown.permanent_control(megakaryocyte_att, erythrocyte_att, verbose=False)
mega_erythro.perturbation_working_colors({"EKLF": True}).cardinality() / CARDINALITY

# WTF

0.0

In [26]:
mega_erythro.working_perturbations(min_robustness=0.99, 
                                             verbose=False,
                                             return_all=False)

[({'Fli1': False, 'CEBPa': False, 'GATA2': False, 'PU1': False},
  ColorSet(cardinality=2963088))]